## Custom Sentiment Analysis 
these first few blocks are me copy-pasting the https://realpython.com/python-nltk-sentiment-analysis/#using-nltks-pre-trained-sentiment-analyzer 
code and learning how it works

In [2]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("Wow, NLTK is really powerful!")

{'neg': 0.0, 'neu': 0.295, 'pos': 0.705, 'compound': 0.8012}

In [3]:
tweets = [t.replace("://", "//") for t in nltk.corpus.twitter_samples.strings()]

In [4]:
from random import shuffle

def is_positive(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia.polarity_scores(tweet)["compound"] > 0

shuffle(tweets)
for tweet in tweets[:10]:
    print(">", is_positive(tweet), tweet)

> False RT @TelePolitics: BBC Question Time: Ed Miliband savaged over his economic record http//t.co/cToppiRYPI
> True In the Porsche paddock at #BudapestGP and I found out #JohnyHerbert from @SkySportsF1 is rolling with us this weekend! :) #PorscheSuperCup
> False RT @davidcbocko: @MichaelLCrick If you're Nick Robinson apparently your duty is to act as a spokesperson for Tory central office.
> True RT @joncstone: ICM snap poll for the #bbcqt special has a solid win for David Cameron: 

44% Cameron
38% Miliband
19% Clegg http//t.co/C3G…
> True @BeeGeeBeard @johnfm15 @OwenJones84 Why are you so keen to work with these "Red Tories" in government?
> True Appreciate your recent Retweets ! @montblancforum @Les_Scop_idf @MiroirSocial @thomasdeysieux @EMauFouX Have a great Friday :)
> True RT @BartchJamie: Amazing how quiet labour get when ed Miliband is on the telly you'd think they were avoiding drawing attention to him,
> True @iamgiant Hope Ryan washed his hair later. Was great night and 

## my turn
Doing the same as above, but now with my corpus instead of tweets

In [5]:
#corpus as list of strings
import pandas as pd
df = pd.read_csv('C:\\Users\Connor\Desktop\\370\letterboxdCorpus\\all.csv')
reviews = df.reviews.tolist()
scoreless = []
for review in reviews:
    scoreless.append(review[5:])
print(scoreless[0:10])

['I highly recommend everyone get drunk with friends and watch this     ', "My introduction to John cena's absolutely delicious looking biceps     ", 'Not fast enough!     ', ' On march 24th 2023 Kanye West said he is no longer anti semitic because of this movie and Jonah Hill     ', "Why are old people having fun they're literally gonna die tomorrow 😭     ", 'Kids movie where dolls fight for their lives after the world ends in a terrifying way   somehow still less scary than happy feet     ', 'Yeah my ass was crying again my ass is always crying     ', 'Worst theater experience of my life fun movie tho     ', 'Wow wow wow wow wow WOW     ', ' Severe lack of solar systems but overall a good time     ']


In [6]:
shuffle(scoreless)
for review in scoreless[:10]:
    print(">", is_positive(review), review)

> True Two thoughts:  1) This is an incredible performance by McConaughey, and one that makes smart use of his existing star text in new, interesting ways; Wooderson gone to seed. Jared Leto is excellent as well.  2) These incredible performances are in a movie that lives into every negative stereotype of Hollywood filmmakers examining a minority issue--in this case the impact of the HIV virus on the gay community in the 1980s, and the FDA's lethargic response to it--from the perspective of a heroic straight white dude. It's not necessarily a bad version of that kind of movie. But it's still that kind of movie.     
> False "We'll have a dalmatian plantation!" or "Kill two for matching clogs!"  So much weird texture here, all bound up in those gorgeous Xeroxed inks, everything has a dusty coating of toner. And what's with the vague anti-capitalist vibe? There's a whole middle section here that takes place in this decrepit mansion, sort of a crumbling bourgeois symbol ruled over by Crue

I honestly don't mind the supposed ~60% accuracy. I do like the results.

## generating top 100s

In [7]:
unwanted = nltk.corpus.stopwords.words("english")
unwanted.extend([w.lower() for w in nltk.corpus.names.words()])
#
#here's my own injection, which makes a list of words in positive reviews and a list
#of negative reviews, so that I don't need to use the nltk.corpus movie reviews
pos_category = []
neg_category = []
for review in scoreless:
    words = review.split()
    if is_positive(review): 
        for word in words:
            pos_category.append(word)
    else:
        for word in words:
            neg_category.append(word)


def skip_unwanted(pos_tuple):
    word, tag = pos_tuple
    if not word.isalpha() or word in unwanted:
        return False
    if tag.startswith("NN"):
        return False
    return True

#here's where it trains off my stuff rather than nltk movie reviews
positive_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(pos_category)
)]
negative_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(neg_category)
)]

In [8]:
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive = {word for word, count in positive_fd.most_common(100)}
top_100_negative = {word for word, count in negative_fd.most_common(100)}
print("POSTIVE:")
print(top_100_positive)
print("NEGATIVE:")
print(top_100_negative)

POSTIVE:
{'satisfied', 'owning', 'revered', 'disappoint', 'prominent', 'thankful', 'kissing', 'joyful', 'easiest', 'inspirational', 'subscribing', 'cutest', 'Subscribing', 'beguiling', 'delighted', 'breathlessly', 'hopeful', 'owed', 'costuming', 'intelligent', 'thrilled', 'approach', 'supportive', 'mined', 'curly', 'exacting', 'superb', 'sharply', 'lovelorn', 'obscure', 'pls', 'ground', 'inclusive', 'hong', 'saturday', 'folks', 'Besides', 'subverts', 'pleasantly', 'remained', 'richly', 'celebrate', 'gimmick', 'impoverished', 'earth', 'blessed', 'minding', 'emerges', 'Easily', 'improvisational', 'entered', 'united', 'documented', 'WANT', 'rebellious', 'kidding', 'unafraid', 'uncharted', 'smile', 'extend', 'perceive', 'hagiographic', 'eps', 'gain', 'transcends', 'grateful', 'cringe', 'panoramic', 'libertarian', 'vibe', 'smooth', 'luckily', 'mandatory', 'seamless', 'Seems', 'Preparing', 'underwhelming', 'enthusiastic', 'captivated', 'sporadically', 'subconsciously', 'award', 'sugary', 'in

So, there are the rough top 100 positive and negative words in my corpus. Apart from a few questionable results here and there, I'd consider it mostly good.

Here's an idea. I'm going to redefine my pos/neg_category variables using the user scores attached to reviews. I'm hoping it gives me more accurate evaluations than the polarity scores

In [9]:
import nltk
from nltk.tokenize import word_tokenize

#8s, 9s, and 10/10s
good_movies = []
#3s, 2s, and 1/10s
bad_movies = []
#var for later
sentences = []
for review in reviews:
    if review.startswith("10") or review.startswith("9") or review.startswith("8"):
        lower = review[5:].lower()
        tokens = word_tokenize(lower)
        for token in tokens:
            good_movies.append(token)
    if review.startswith("1/") or review.startswith("2") or review.startswith("3"):
        lower = review[5:].lower()
        tokens = word_tokenize(lower)
        for token in tokens:
            bad_movies.append(token)
    lower2 = review[5:].lower()
    tokens2 = word_tokenize(lower2)
    sentences.append(tokens2)
print(bad_movies[0:50])

['i', 'highly', 'recommend', 'everyone', 'get', 'drunk', 'with', 'friends', 'and', 'watch', 'this', 'my', 'introduction', 'to', 'john', 'cena', "'s", 'absolutely', 'delicious', 'looking', 'biceps', 'i', 'like', 'that', 'one', 'robert', 'de', 'niro', 'comedy', 'where', 'he', 'shows', 'his', 'cock', 'and', 'fucks', 'like', 'crazy', 'this', 'is', 'not', 'that', 'one', 'tho', 'i', 'know', 'a', 'guy', 'named', 'aksel']


I'm too lazy to go up and make a function

In [10]:
#now processing new stuff see line 4 and 8
positive_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(good_movies)
)]
negative_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(bad_movies)
)]
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive = {word for word, count in positive_fd.most_common(100)}
top_100_negative = {word for word, count in negative_fd.most_common(100)}
print("POSTIVE:")
print(top_100_positive)
print("NEGATIVE:")
print(top_100_negative)

POSTIVE:
{'allows', 'bodily', 'subscribing', 'gothic', 'inherent', 'grounded', 'vivid', 'detailed', 'stronger', 'scariest', 'warm', 'devastating', 'empathetic', 'tiny', 'noir', 'moves', 'masterful', 'vital', 'sudden', 'ep', 'fantastical', 'adds', 'trapped', 'rewatch', 'pulls', 'grew', 'shaking', 'gleefully', 'tense', 'internal', 'sees', 'wrapped', 'neon', 'delicate', 'exquisite', 'twin', 'sensual', 'familial', 'experimental', 'atmosphere', 'marvelous', 'loving', 'elegant', 'gorgeously', 'builds', 'detective', 'apocalyptic', 'melancholic', 'lying', 'expertly', 'harsh', 'layered', 'harrowing', 'claustrophobic', 'flawed', 'grateful', 'ghostly', 'phenomenal', 'indelible', 'fitting', 'finds', 'hysterical', 'showcasing', 'unexpected', 'surreal', 'uses', 'magnificent', 'underrated', 'breaking', 'nightmarish', 'shown', 'similar', 'touching', 'essential', 'dreamy', 'vividly', 'classical', 'gentle', 'allow', 'adding', 'fractured', 'vicious', 'beautifully', 'flows', 'existential', 'impeccable', '

Even cooler results. Although maybe affected by a smaller sample size, since I'm seeing the infrequently used "fuckahhhhhhhhh" in the negative.

So my best idea to find more stuff with word embeddings is to process my positive and negative words and see what pops up frequently in .most_similar calls

In [11]:
from gensim.models import Word2Vec
print(len(sentences))
model = Word2Vec(sentences = sentences,
                 window = 2, 
                 min_count =1,
                 vector_size = 5)

25344


In [23]:
#similar_words = model.wv.most_similar('sensual', topn= 10)
# ^ list of tuples. first word = [0][0]
#making a dictionary in the (word, count) format
near_positive = {}
for word in top_100_positive:
    sim = model.wv.most_similar(word, topn= 500)
    for i in range(500):
        synonym = sim[i][0]
        #print(word)
        if synonym in near_positive:
            near_positive[synonym] += 1
        else:
            near_positive[synonym] = 1
sorted_pos = {k: v for k, v in sorted(near_positive.items(), key=lambda item: item[1], reverse=True)}
print("WORDS \"SIMILAR TO\" POSITIVE WORDS:")
message = ""
for key, value in list(sorted_pos.items())[:200]:
    message += f"{key}:{value}  "
print(message)

WORDS "SIMILAR TO" POSITIVE WORDS:
natural:32  throwback:31  avatar:30  pulp:30  fits:30  lo-fi:30  petty:30  anonymous:29  thrills:29  blurring:29  dtv:29  zac:28  cavities:28  dangerous:28  graphic:28  purity:27  conventions:27  ambitions:27  gruesome:27  zooms:27  incoherent:27  paramedics:27  videos:27  promises:27  awkward:27  famous:27  chase:27  caper:27  styles:26  elite:26  exercise:26  vintage:26  kaiju:26  effect:26  heist:26  sincere:26  stature:26  pageantry:26  grisly:26  conspiracy:25  network:25  grand:25  empathetic:25  high-concept:25  chamber:25  vivid:25  post-war:25  worthy:25  exposition:25  barren:25  class-divided:25  victims:25  meeting:25  relative:25  bedroom:24  coda:24  superficiality:24  expression:24  patience:24  cartoon:24  reformed:24  thematic:24  institutionalized:24  architecture:24  becoming:24  interview:24  erotic:24  infinite:24  biker:24  slipping:24  french:24  affair:24  destruction:24  random:24  canvas:24  gentle:24  staggering:24  ex-sovie

In [24]:
near_negative = {}
for word in top_100_negative:
    sim = model.wv.most_similar(word, topn= 500)
    for i in range(500):
        synonym = sim[i][0]
        #print(word)
        if synonym in near_negative:
            near_negative[synonym] += 1
        else:
            near_negative[synonym] = 1
sorted_neg = {k: v for k, v in sorted(near_negative.items(), key=lambda item: item[1], reverse=True)}
print("WORDS \"SIMILAR TO\" NEGATIVE WORDS:")
message = ""
for key, value in list(sorted_neg.items())[:200]:
    message += f"{key}:{value}  "
print(message)

WORDS "SIMILAR TO" NEGATIVE WORDS:
slow-motion:10  hardcore:10  ride:10  utterly:10  dirty:9  microcosm:9  unearths:9  geography:9  introduce:9  demonstrably:9  slow:9  ragged:9  valuable:9  spy:9  vibe:9  little-seen:9  cosplay:9  derivative:9  confident:9  overlong:9  corny:9  standout:9  half-quizzical:9  exhausting:9  issue:9  endlessly:9  irony:9  cliche:8  gore:8  oblivious:8  atmospheric:8  mines:8  zach:8  featuring:8  substituting:8  buñuel:8  demon:8  nails:8  feminist:8  pretentious:8  following:8  unnerving:8  lacking:8  ugly:8  seriousness:8  annoying:8  signed:8  demand:8  grasp:8  odd:8  mihai:8  unsurprisingly:8  admirable:8  well-meaning:8  co-opted:8  brando:8  realistic:8  slicker:8  motive:8  winking:8  enlightening:8  d'abo:8  unsettled:8  adorable:8  paced:8  artist:8  jettisons:8  bud:8  assistant:8  intertwined:8  dangerous:8  heavily:8  rich:8  operate:8  vital:8  crafted:8  playing:8  executed:8  monotonous:8  racist:8  measure:8  falls:8  needlessly:8  quiet:

These results kinda suck, but improve some as i turn up the topn parameter. I imagine as the count values decrease, the less reliable the words get.